In [1]:
import torch
import torch.nn.functional as F
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import os
import pickle

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
file = open("datasets/a_room_with_a_view.txt", "r", encoding = "utf8")
lines = []

for i in file:
    lines.append(i)
    
print("The first line: ", lines[0])

The first line:  ﻿The Project Gutenberg eBook of A Room With A View, by E. M. Forster



### Romove escape characters and symbols

In [4]:
data = ""

for i in lines:
    data = ' '. join(lines)
    
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '')
data[:360]

import string

translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
data = data.translate(translator)

data[:500]

'The Project Gutenberg eBook of A Room With A View  by E  M  Forster  This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever  You may copy it  give it away or re use it under the terms of the Project Gutenberg License included with this eBook or online at www gutenberg org  If you are not located in the United States  you will have to check the laws of the country where you are located before using '

### Tokenization

In [5]:
from transformers import XLNetTokenizer

In [6]:
# text = "I am a good boy with handrads of friends"
tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")
tokenized = tokenizer.tokenize(data)
ids = tokenizer.convert_tokens_to_ids(tokenized)

In [7]:
print(tokenized[512:532])
print(tokenizer.convert_tokens_to_ids(tokenized[512:532]))

['▁and', '▁red', '▁bottles', '▁of', '▁wine', '▁that', '▁ran', '▁between', '▁the', '▁English', '▁people', '▁at', '▁the', '▁portraits', '▁of', '▁the', '▁late', '▁Queen', '▁and', '▁the']
[21, 1170, 10596, 20, 2680, 29, 1662, 161, 18, 897, 104, 38, 18, 16738, 20, 18, 471, 3631, 21, 18]


In [8]:
vocab_size = tokenizer.vocab_size

In [9]:
ids_arr = np.array(ids)

sequences = np.empty((0,4), int)
for i in range(3, len(ids)):
    words = np.expand_dims(ids_arr[i-3:i+1], axis=0)
    sequences = np.append(sequences, words,axis=0)

In [10]:
X = []
Y = []
for i in sequences:
    X.append(i[0:3])
    Y.append(i[3])
X = np.array(X)
Y = np.array(Y)

In [11]:
Y = np.eye(vocab_size)[Y]

### parameters

In [12]:
input_size = 3
hidden_size = 256
num_layers = 2
num_classes = vocab_size
sequence_length = vocab_size
learning_rate = 0.005
batch_size = 64
num_epochs = 3

### RNN model

In [13]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size * sequence_length, num_classes) ### softmax

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagate LSTM
        out, _ = self.rnn(x, h0)
        out = out.reshape(out.shape[0], -1)

        # Decode the hidden state of the last time step
        out = self.fc(out)
        return out

In [14]:
# Load Data
num_train = len(X)*9//10
print(num_train)
train_x = X[:num_train]
test_x = X[num_train:]

train_y = Y[:num_train]
test_y = Y[num_train:]


train = torch.utils.data.TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
train_loader = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True)
test = torch.utils.data.TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))
test_loader = torch.utils.data.DataLoader(test, batch_size=64, shuffle=True)

78675


In [ ]:
# Initialize network (try out just using simple RNN, or GRU, and then compare with LSTM)
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # Get data to cuda if possible
#         data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent update step/adam step
        optimizer.step()

# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0

    # Set model to eval
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    # Toggle model back to train
    model.train()
    return num_correct / num_samples


print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")